In [1]:
from lib import *

In [2]:
train_set = pd.read_csv('train.csv')
val_set = pd.read_csv('val.csv')
test_set = pd.read_csv('test.csv')

In [3]:
class GlobalCovModel(CovarianceModel):
    def __init__(self):
        self.cov = None

    def train(self, train_set: pd.DataFrame):
        self.cov = np.cov(df_residuals(train_set).T, ddof=1)
        print('Training complete')
        print(pretty_cov(self.cov))

    def estimate(self, long: float, lat: float, intensity: float) -> np.ndarray:
        return self.cov

print(f"\nGlobalCovModel {GlobalCovModel().assess_geo_mean_log_likelihood(train_set, test_set)}")

Training complete
[[0.398, 0.010], [0.010, 0.268]]

GlobalCovModel -4.200237183756096


In [4]:
class BinnedCovModel(CovarianceModel):
    def __init__(self):
        self.covs = {}

    def train(self, train_set: pd.DataFrame):
        for north in [True, False]:
            correct_hemi = train_set[train_set.lat > 0] if north else train_set[train_set.lat < 0]
            for inten in [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0]:
                subset = correct_hemi[correct_hemi.intensity == inten]
                self.covs[north, inten] = np.cov(df_residuals(subset).T, ddof=1)
        print('Training complete')
        for north, inten in self.covs:
            print(f"{'N' if north else 'S'} {inten}: {pretty_cov(self.covs[north, inten])}")

    def estimate(self, long: float, lat: float, intensity: float) -> np.ndarray:
        return self.covs[lat > 0, intensity]

print(f"\nBinnedCovModel {BinnedCovModel().assess_geo_mean_log_likelihood(train_set, test_set)}")

Training complete
N -5.0: [[0.933, 0.024], [0.024, 0.403]]
N -4.0: [[1.863, 0.133], [0.133, 0.580]]
N -3.0: [[0.292, 0.014], [0.014, 0.241]]
N -2.0: [[0.541, 0.055], [0.055, 0.281]]
N -1.0: [[0.327, 0.022], [0.022, 0.261]]
N 0.0: [[0.340, 0.042], [0.042, 0.238]]
N 1.0: [[0.335, 0.031], [0.031, 0.220]]
N 2.0: [[0.288, 0.041], [0.041, 0.201]]
N 3.0: [[0.223, 0.028], [0.028, 0.182]]
N 4.0: [[0.194, 0.031], [0.031, 0.168]]
N 5.0: [[0.184, 0.052], [0.052, 0.165]]
S -5.0: [[0.650, -0.052], [-0.052, 0.413]]
S -4.0: [[0.793, -0.179], [-0.179, 0.547]]
S -3.0: [[0.301, -0.017], [-0.017, 0.257]]
S -2.0: [[0.462, -0.079], [-0.079, 0.291]]
S -1.0: [[0.346, -0.029], [-0.029, 0.277]]
S 0.0: [[0.344, -0.038], [-0.038, 0.277]]
S 1.0: [[0.304, -0.039], [-0.039, 0.234]]
S 2.0: [[0.308, -0.029], [-0.029, 0.221]]
S 3.0: [[0.251, -0.035], [-0.035, 0.210]]
S 4.0: [[0.219, -0.029], [-0.029, 0.171]]
S 5.0: [[0.186, -0.045], [-0.045, 0.237]]

BinnedCovModel -3.9268145071588076
